In [1]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")
include("c:/Users/lisah/Documents/Repos/ma-code/src/src.jl")
using Distributions, Plots, KernelDensity, DataFrames, Optim, ForwardDiff, LinearAlgebra

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


Notes on usage:
- change values in cell below, then run all
- t_end is time point until which system is simulated
- t_pt_sample_dens is time point at which we evaluate sample density
- n is number of samples

In [2]:
n = 100
t_end = 50.0
t_pt_sample_dens = 5
noise = 0.0;

In [3]:
# don't change
t_fixed = true
M = Int(t_end * 10) # plot M points
t_pt_sample_dens = t_pt_sample_dens  * 10;

In [4]:
# fixed parameter values
a = mean([0.94,2.81]) # suggested by Klausmeier paper
m = 0.45 # suggested by Klausmeier paper
n0 = 1.0 # no source
w0 = 1.0; # no source

In [5]:
# create true data observations
hprm = Src.Hyperprm(w0,n0,a,m,M,noise)
data_obs = Src.sol_klausmeier(hprm)
data_obs = Src.randomize_data(data_obs, noise);

## Determine posteriors / Inverse UQ

We use the Fisher approximation of the posterior, i.e. the posterior is of Gaussian form, with MLE = mean and FIM^(-1) = covariance matrix

### a

In [6]:
function compute_ll_a(x, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    a = x[1]
    hprm = Src.Hyperprm(hprm.w0, hprm.n0, a, hprm.m, hprm.M, hprm.noise)
    pred_val = Src.sol_klausmeier(hprm; t_fixed=t_fixed, t_end=t_end, t_step=t_step)
    if hprm.noise == 0.0
        ll = -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    else
        ll = -0.5 * 1/hprm.noise * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * 1/hprm.noise * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    end
    return ll
end

function compute_mle_a(hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    result = optimize(x -> -compute_ll_a(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), 0.94, 2.81, Brent()) # initialize optimization at true prm values s.th. global min is found
    return Optim.minimizer(result), Optim.converged(result)
end

function compute_fi_a(eval_pt::Vector{Float64}, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    H = ForwardDiff.hessian(x -> compute_ll_a(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), eval_pt)
    return tr(-H)
end

compute_fi_a (generic function with 1 method)

In [7]:
mle_a = compute_mle_a(hprm, data_obs, t_fixed=t_fixed, t_end=t_end)[1]
fim_a = compute_fi_a([mle_a], hprm, data_obs, t_fixed=t_fixed, t_end=t_end)
cov_a = inv(fim_a);

In [11]:
(mle_a, cov_a)

(1.9049714155607247, 0.0003811043671003655)

### m

In [8]:
function compute_ll_m(x, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    m = x[1]
    hprm = Src.Hyperprm(hprm.w0, hprm.n0, hprm.a, m, hprm.M, hprm.noise)
    pred_val = Src.sol_klausmeier(hprm; t_fixed=t_fixed, t_end=t_end, t_step=t_step)
    if hprm.noise == 0.0
        ll = -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    else
        ll = -0.5 * 1/hprm.noise * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * 1/hprm.noise * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    end
    return ll
end

function compute_mle_m(hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    result = optimize(x -> -compute_ll_m(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), 0.45-0.45*2, 0.45+0.45*2, Brent()) # initialize optimization at true prm values s.th. global min is found
    return Optim.minimizer(result), Optim.converged(result)
end

function compute_fi_m(eval_pt::Vector{Float64}, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    H = ForwardDiff.hessian(x -> compute_ll_m(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), eval_pt)
    return tr(-H)
end

compute_fi_m (generic function with 1 method)

In [12]:
mle_m = compute_mle_m(hprm, data_obs, t_fixed=t_fixed, t_end=t_end)[1]
fim_m = compute_fi_m([mle_m], hprm, data_obs, t_fixed=t_fixed, t_end=t_end)
cov_m = inv(fim_m);

In [13]:
(mle_m,cov_m)

(0.44439784973557933, 2.155632459400449e-5)

### n0

In [27]:
function compute_ll_n0(x, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    n0 = x[1]
    hprm = Src.Hyperprm(hprm.w0, n0, hprm.a, hprm.m, hprm.M, hprm.noise)
    pred_val = Src.sol_klausmeier(hprm; t_fixed=t_fixed, t_end=t_end, t_step=t_step)
    if hprm.noise == 0.0
        ll = -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    else
        ll = -0.5 * 1/hprm.noise * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * 1/hprm.noise * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    end
    return ll
end

function compute_mle_n0(hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    result = optimize(x -> -compute_ll_n0(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), n0-0.1, n0+0.1, Brent()) # initialize optimization at true prm values s.th. global min is found
    return Optim.minimizer(result), Optim.converged(result)
end

function compute_fi_n0(eval_pt::Vector{Float64}, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    H = ForwardDiff.hessian(x -> compute_ll_n0(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), eval_pt)
    return tr(-H)
end

compute_fi_n0 (generic function with 1 method)

In [28]:
mle_n0 = compute_mle_n0(hprm, data_obs, t_fixed=t_fixed, t_end=t_end)[1]
fim_n0 = compute_fi_n0([mle_n0], hprm, data_obs, t_fixed=t_fixed, t_end=t_end)
cov_n0 = inv(fim_n0);

In [29]:
(mle_n0, cov_n0)

(1.0999999753306569, 0.007090625617170169)

### w0

In [24]:
function compute_ll_w0(x, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    w0 = x[1]
    hprm = Src.Hyperprm(w0, hprm.n0, hprm.a, hprm.m, hprm.M, hprm.noise)
    pred_val = Src.sol_klausmeier(hprm; t_fixed=t_fixed, t_end=t_end, t_step=t_step)
    if hprm.noise == 0.0
        ll = -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    else
        ll = -0.5 * 1/hprm.noise * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * 1/hprm.noise * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    end
    return ll
end

function compute_mle_w0(hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    result = optimize(x -> -compute_ll_w0(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), w0-0.1, w0+0.1, Brent()) # initialize optimization at true prm values s.th. global min is found
    return Optim.minimizer(result), Optim.converged(result)
end

function compute_fi_w0(eval_pt::Vector{Float64}, hprm::Src.Hyperprm, true_val::DataFrame; t_fixed::Bool=false, t_end::Float64=50.0, t_step::Float64=1.0)
    H = ForwardDiff.hessian(x -> compute_ll_w0(x, hprm, true_val; t_fixed=t_fixed, t_end=t_end, t_step=t_step), eval_pt)
    return tr(-H)
end

compute_fi_w0 (generic function with 1 method)

In [25]:
mle_w0 = compute_mle_w0(hprm, data_obs, t_fixed=t_fixed, t_end=t_end)[1]
fim_w0 = compute_fi_w0([mle_w0], hprm, data_obs, t_fixed=t_fixed, t_end=t_end)
cov_w0 = inv(fim_w0);

In [26]:
(mle_w0, cov_w0)

(1.0999999753306569, 0.05635193419389328)

## Local sensitivity analysis on posteriors